# 1_PreProcessing_Input_Files.ipynb

Take in the Valuations files month by month.

 For each month:
  - process each file by removing all rows with 0 units (these have no monry invested in a fund, may be non-active).
  - for the paxus file, add headers.
  - for both, remove all cilumns except teh policy number and teh fund code (DUB_NAME)
  - for paxus, drop all records where VALID_FLAG is not TRUE
  - for both, add a RUN_MONTH column and populate it with teh month in question.
 
 Then we end up with a dataframe for each system with the same structure.  Merge and sort them and now we have a single, clean source for that month.  This is written to a csv file for later processing.


## Import libraries

In [1]:
# importing the librarIES
import os                           # For file processing.
import csv
import pandas as pd            # For dataframes.
from pathlib import Path
import numpy  as np                 
import pyodbc                       # For connecting to SQL Server
import datetime
import numpy

import psutil 
import gc

import hashlib

## Set all runtime variables.

In [2]:
e = datetime.datetime.now()
print ("Current date and time = %s" % e)
print ("Current memory available = ",psutil.virtual_memory().available * 100 / psutil.virtual_memory().total)
# Drop all cols except 'POL_NUMB','DUB_NAME','NO_UNITS_DIS'
#cols_to_keep = ['POL_NUMB','DUB_NAME','NO_UNITS_DIS']
cols_to_keep = ['POL_NUMB','DUB_NAME','API_TRANCHE_EXT']

colspecs = [(0, 2), (2,12), (12,14), (15,25), (26,34), (34,52), (52,53), (53,58), (58,77), 
            (78,96), (96,103), (104,112), (113,120), (121,128), (129,136), (137,144), (144,145)
           ]

names=['SUB_FUND','POL_NUMB','INV_COV_EXT','API_TRANCHE_EXT', 'TR_COMM_DATE_EXT', 'TR_ADCON_EXT','FUND_TYPE','DUB_NAME',
'NO_UNITS_NOM','NO_UNITS_DIS','TR_ENT_DATE','CINC_CONVERSION_DATE','CVF_FACTOR','DVF_FACTOR','BVF_FACTOR','SVF_FACTOR',
'VALID_FLAG']

aggregation_functions = {'API_TRANCHE_EXT': 'sum','DUB_NAME': 'sum'}

# Set a value for the home folder.
home_folder = ".\data"

# Set values for the various paths.
input_path = home_folder + "\input"
print("input_path",input_path)
output_path = home_folder + "\output"
print("output_path",output_path)
merged_path = ".\data\merged"
print("merged_path",merged_path)

# Set values for the various file extension types.
paxus_in_file_type = ".DTA"
isuite_file_type = ".csv"
out_file_type = ".csv"

# Set values for the various file prefixes.
paxus_file_prefix = "VAL_FUNDGER_"
isuite_file_prefix = "val-fundger-"
cleaned_file_prefix = "Clean_"
merged_file_prefix = "Fund_Investment_History_"


Current date and time = 2023-04-22 12:04:29.368317
Current memory available =  38.46522760671952
input_path .\data\input
output_path .\data\output
merged_path .\data\merged


## Function to process the Paxus VAL_FUNDGER.DTA file and the iSuite val-fundger.csv file.

 Merge the data, write the file and return a dataframe

In [3]:
def Wrangle_the_data(yearmonth):
    
    # 1.  Read the Paxus valuation file into a dataframe.
    #######################################################################
    # Set the file names using the year passed in and join various path components
    in_file  =   paxus_file_prefix + yearmonth + paxus_in_file_type
    in_file_path = os.path.join(input_path, in_file)
    # Read the Paxus file into a dataframe
    df_in=pd.read_fwf(in_file_path, skiprows=0, skipfooter=0, colspecs=colspecs, names=names)
    print("Paxus in_file_path-",in_file_path,len(df_in))

    # 2.  Perform data cleansing and pre-processing for the file dataframe.
    #######################################################################
    # Only keep rows with a FUND_TYPE = 'A' AND NO_UNITS_DIS > 0
    df_out = df_in.loc[~((df_in['FUND_TYPE'] != 'A') | (df_in['NO_UNITS_DIS'] ==0) )]
    # Sort by policy number and then Fund code to a new dataframe.
    df_out = df_out.sort_values(['POL_NUMB','DUB_NAME']).reset_index(drop=True)
    # Only keep rows with a non NULL VALID_FLAG.
    df_out = df_out.dropna(axis=0, subset=['VALID_FLAG'])
    # Drop all cols except 'POL_NUMB','DUB_NAME','NO_UNITS_DIS'
    df_out = df_out[cols_to_keep]
    # Stamp the record with teh month of extract
    df_out['RUN_MONTH'] = yearmonth

    # 3. Write out the enhanced data for future use.
    ################################################
    # Set the file names using the year passed in and join various path components
    out_file  =  cleaned_file_prefix + paxus_file_prefix + yearmonth + out_file_type
    out_file_path = os.path.join(output_path, out_file)
    print("Paxus out_file_path-",out_file_path,len(df_out))
    # Write the dataframe to a csv file. 
    df_out.to_csv(out_file_path, encoding='utf-8', index=False)

    # 4. Aggregate the sorted fund codes into one string for month on month comparison.
    ###################################################################################
    df_agg_p = df_out.groupby(df_out['POL_NUMB']).aggregate(aggregation_functions)
    df_agg_p['RUN_MONTH'] = yearmonth

############################################     ISUITE PROCESSING        ##############################################
    # 1.  Read the iSuites valuation file into a dataframe.
    #######################################################################
    # Set the file names using the year passed in and join various path components
    in_file  =   isuite_file_prefix + yearmonth + isuite_file_type
    in_file_path = os.path.join(input_path, in_file)
    # Read the iSuite file into a dataframe
    df_in = pd.read_csv(in_file_path, delimiter=';',low_memory=False, header=0)
    print("iSuite in_file_path-",in_file_path,len(df_in))

    # 2.  Perform data cleansing and pre-processing for the file dataframe.
    #######################################################################
    # Only keep rows with a non NULL VALID_FLAG.
    df_out = df_in.dropna(axis=0, subset=['NO_UNITS_NOM'])
    # Drop all cols except 'POL_NUMB','DUB_NAME','NO_UNITS_NOM'
    df_out = df_out[cols_to_keep]
    # Sort by policy number and then Fund code to a new dataframe.
    df_out = df_out.sort_values(['POL_NUMB','DUB_NAME']).reset_index(drop=True)
    # Stamp the record with teh month of extract
    df_out['RUN_MONTH'] = yearmonth

    # 3. Write out the enhanced data for future use.
    ################################################
    # Write to a csv file and use index=false to drop teh index from the dataframe
    # Set the file names using the year passed in and join various path components
    out_file  =  cleaned_file_prefix + isuite_file_prefix + yearmonth + out_file_type
    out_file_path = os.path.join(output_path, out_file)
    df_out.to_csv(out_file_path, encoding='utf-8', index=False)
    print("iSuite out_file_path-",out_file_path,len(df_out))
    
    # 4. Aggregate the sorted fund codes into one string for month on month comparison.
    ###################################################################################
    df_agg_i = df_out.groupby(df_out['POL_NUMB']).aggregate(aggregation_functions)
    df_agg_i['RUN_MONTH'] = yearmonth
    
    # 5. Merge the Paxus and iSuite dataframes, merge and sort them write to a csv file.
    ###################################################################################
    #    Use index=false to drop teh index from the dataframe
    # Merge and sort the two dataframes
    df_merged = pd.concat([df_agg_p, df_agg_i], ignore_index=False, sort=True)

    # Write to a csv file and use index=false to drop teh index from the dataframe
    merged_file  =  merged_file_prefix + yearmonth + out_file_type
    merged_file_path = os.path.join(merged_path, merged_file)
    print("Merged_file_path-",merged_file_path)
    df_merged.to_csv(merged_file_path, encoding='utf-8', index=True)
    
    print("How many rows in the Paxus dataframe?  ",len(df_agg_p))
    print("How many rows in the iSuite dataframe? ",len(df_agg_i))
    print("How many rows in the merged dataframe? ",len(df_merged) )
    return df_merged    

## Calling function, allowing a year to be passed.

In [4]:
def Process_files_for_a_month(yearmonth):

    print("Running for",yearmonth)
    print("==================")
    df_Valuations  = Wrangle_the_data(yearmonth)
    return df_Valuations   

## Process all of the input files, month by month.


In [5]:
Process_files_for_a_month('202201')
Process_files_for_a_month('202202')
Process_files_for_a_month('202203')
Process_files_for_a_month('202204')
Process_files_for_a_month('202205')
Process_files_for_a_month('202206')
Process_files_for_a_month('202207')
Process_files_for_a_month('202208')
Process_files_for_a_month('202209')
Process_files_for_a_month('202210')
Process_files_for_a_month('202211')
Process_files_for_a_month('202212')
Process_files_for_a_month('202301')
Process_files_for_a_month('202302')
Process_files_for_a_month('202303')

Running for 202201
Paxus in_file_path- .\data\input\VAL_FUNDGER_202201.DTA 5436796
Paxus out_file_path- .\data\output\Clean_VAL_FUNDGER_202201.csv 753727
iSuite in_file_path- .\data\input\val-fundger-202201.csv 109855
iSuite out_file_path- .\data\output\Clean_val-fundger-202201.csv 55546
Merged_file_path- .\data\merged\Fund_Investment_History_202201.csv
How many rows in the Paxus dataframe?   53641
How many rows in the iSuite dataframe?  40635
How many rows in the merged dataframe?  94276
Running for 202202
Paxus in_file_path- .\data\input\VAL_FUNDGER_202202.DTA 5440019
Paxus out_file_path- .\data\output\Clean_VAL_FUNDGER_202202.csv 753368
iSuite in_file_path- .\data\input\val-fundger-202202.csv 115955
iSuite out_file_path- .\data\output\Clean_val-fundger-202202.csv 59000
Merged_file_path- .\data\merged\Fund_Investment_History_202202.csv
How many rows in the Paxus dataframe?   53442
How many rows in the iSuite dataframe?  43532
How many rows in the merged dataframe?  96974
Running for 

,API_TRANCHE_EXT,DUB_NAME,RUN_MONTH
POL_NUMB,,,
001045160#,132935.50,TSDGBTSDITTSEETTSSITTSYGB,202303
001045178#,17894.80,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,202303
001046154#,7669.30,TSESS,202303
001047062#,2556.40,TSDGB,202303
001049026#,2556.40,TSESD,202303
...,...,...,...
8157682K,6768.00,XBEAAXBEAS,202303
8157702V,1200.00,5024A,202303
8157705J,2400.00,XBEAAXBEAS,202303
